# DataQuest.io guided project about building a database for crime reports
GOALS: Practice following concepts:
    -How to create a database and manage database roles
        =Database named crimes_db
        =Groups of "readonly" and "readwrite"
    -How to create database schemas and tables with proper datatypes
        =Schema named "crimes" with table named "boston_crimes"
    -How to load dataf from CSV files into dataabse tables.

Included data: boston.csv
columns = incident_number, offense_code, description, date, day_of_the_week, lat, long

# Connecting to the Dataquest Database
# Creating new table

In [1]:
import psycopg2

In [2]:
#need to connet to an initial database. Using 'dq'
conn = psycopg2.connect(dbname='dq', user='dq')
cur = conn.cursor()
# Create the crimes_db database
# Turn on autocommit to create a postgres database
cur.execute("SELECT datname FROM pg_database;")
databases = cur.fetchall()
databases = [row[0] for row in databases]
if 'crimes_db' not in databases:
    print("creating new database: crimes_db")
    conn.rollback()
    conn.autocommit = True
    cur.execute("CREATE DATABASE crimes_db;")
else:
    print("database crimes_db already exists")
conn.close()
#Connect to the new database
conn = psycopg2.connect(dbname='crimes_db', user='dq')
cur = conn.cursor()
#Creating new schema
cur.execute("SELECT schema_name FROM information_schema.schemata;")
schemas = cur.fetchall()
schemas = [row[0] for row in schemas]
if 'crimes' not in schemas:
    print("Creating new schema: crimes")
    cur.execute("CREATE SCHEMA crimes;")
else:
    print("Schema crimes already exists")
conn.commit()
conn.close()

database crimes_db already exists
Schema crimes already exists


In [3]:
import csv
with open('boston.csv', 'r') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [4]:
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [5]:
first_row

['1',
 '619',
 'LARCENY ALL OTHERS',
 '2018-09-02',
 'Sunday',
 '42.35779134',
 '-71.13937053']

In [6]:
# Create a repeatable way to identify proper datatypes.
def get_col_set(csv_filename, col_index):
    """
    Retrieves distinct values from a specified column in a CSV file.

    Args:
        csv_filename (str): The name of the CSV file.
        col_index (int): The index of the column (0-based) to extract values from.

    Returns:
        set: A set containing unique values from the specified column.
    """
    unique_values = set()

    with open(csv_filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if col_index < len(row):
                unique_values.add(row[col_index])

    return unique_values

# Example usage:
# csv_file_name = 'boston.csv'
# column_index = 2  # Replace with the desired column index
# result_set = get_col_set(csv_file_name, column_index)
# print(result_set)

In [7]:
# Explore the data we have
# Number of unique values: which columns to give enumerated datatype
# Longest unique value: Varchar length to put
for x in range(7):
    uniques = get_col_set('boston.csv', x)
    num_uniques = len(uniques)
    max_len = 0
    for u in uniques:
        if len(u) > max_len:
            max_len = len(u)
    print('Column index {} has {} unique values'.format(x, num_uniques))
    print('Longest text in index {} is {}'.format(x, max_len))

Column index 0 has 298330 unique values
Longest text in index 0 is 15
Column index 1 has 220 unique values
Longest text in index 1 is 12
Column index 2 has 240 unique values
Longest text in index 2 is 58
Column index 3 has 1178 unique values
Longest text in index 3 is 10
Column index 4 has 8 unique values
Longest text in index 4 is 15
Column index 5 has 18178 unique values
Longest text in index 5 is 11
Column index 6 has 18178 unique values
Longest text in index 6 is 12


From the above number of unique values, column 4 is a good candidate for enumerated values.

Below, we want to find the length of the longest Description.
We know description is index 2 from the list of column headers

In [8]:
# Get the longest description (index 2)
desc_uniqs = get_col_set('boston.csv', 2)
max_len = 0
for desc in desc_uniqs:
    if len(desc) > max_len:
        max_len = len(desc)
print(max_len)

58


In [9]:
week_uniqs = get_col_set('boston.csv', 4)
week_uniqs

{'Friday',
 'Monday',
 'Saturday',
 'Sunday',
 'Thursday',
 'Tuesday',
 'Wednesday',
 'day_of_the_week'}

Max length of the description field is 58 characters. For safety, going to set the varchar(100) to allow for a bit more flexability.

So we're making the following columns:
incident_id from 'incident_number', type=bigint PRIMARY KEY
offense_code from 'offense_code', type=integer
description from 'description', type=varchar(100)
date from 'date', type=date
day_of_the_week from 'day_of_the_week', type=enumeration
lat from 'lat', type=double precision
long from 'long', type=double precision

In [13]:
#Create the table in the crimes schema if there is no crimes table yet.
conn = psycopg2.connect(dbname='crimes_db', user='dq')
cur = conn.cursor()
cur.execute("SELECT * FROM information_schema.tables WHERE table_schema = 'crimes';")
crime_tables = cur.fetchall()
crime_tables = [row[0] for row in crime_tables]
if 'boston_crimes' not in crime_tables:
    cur.execute("""CREATE TYPE days_enum AS ENUM ('Monday',
'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');""")
    # Making the boston_crimes table in the crimes schema
    table_statement = """CREATE TABLE crimes.boston_crimes (
    incident_id bigint PRIMARY KEY,
    offense_code integer,
    description varchar(100),
    date date,
    day_of_the_week days_enum,
    lat double precision,
    long double precision);"""
    cur.execute(table_statement)
    print("Created crimes.boston_crimes")
#     test_delete = """DROP TABLE IF EXISTS crimes.test;"""
#     cur.execute(test_delete)
    conn.commit()
conn.close()

Created crimes.boston_crimes


In [11]:
conn = psycopg2.connect(dbname='crimes_db', user='dq')
cur = conn.cursor()
cur.execute("SELECT * FROM information_schema.columns WHERE table_schema = 'information_schema' AND table_name   = 'tables';")
print(cur.fetchall())
conn.close()

[('crimes_db', 'information_schema', 'tables', 'table_catalog', 1, None, 'YES', 'character varying', None, 1073741824, None, None, None, None, None, None, None, None, None, None, None, None, 'crimes_db', 'information_schema', 'sql_identifier', 'crimes_db', 'pg_catalog', 'varchar', None, None, None, None, '1', 'NO', 'NO', None, None, None, None, None, None, 'NEVER', None, 'NO'), ('crimes_db', 'information_schema', 'tables', 'table_schema', 2, None, 'YES', 'character varying', None, 1073741824, None, None, None, None, None, None, None, None, None, None, None, None, 'crimes_db', 'information_schema', 'sql_identifier', 'crimes_db', 'pg_catalog', 'varchar', None, None, None, None, '2', 'NO', 'NO', None, None, None, None, None, None, 'NEVER', None, 'NO'), ('crimes_db', 'information_schema', 'tables', 'table_name', 3, None, 'YES', 'character varying', None, 1073741824, None, None, None, None, None, None, None, None, None, None, None, None, 'crimes_db', 'information_schema', 'sql_identifier', 

In [ ]:
def is_table_empty(table_name):
    try:
        # Connect to the database (replace with your database credentials)
        conn = psycopg2.connect(dbname='crimes_db', user='dq')
        cur = conn.cursor()
        cur.execute("SELECT EXISTS(SELECT 1 FROM %s LIMIT 1);", (table_name))
        result = cur.fetchone()[0]  # Fetch the boolean value
        return not result  # Return True if empty, False otherwise

    except psycopg2.Error as error:
        raise Exception(f"Error checking table emptiness: {error}") from error

In [14]:
# Add the file into the database
conn = psycopg2.connect(dbname='crimes_db', user='dq')
cur = conn.cursor()
if is_table_empty('crimes.boston_crimes'):
    print("adding boston.csv into crimes.boston_crimes")
    with open('boston.csv', 'r') as f:
        cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
    conn.commit()
else:
    print("boston_crimes already has data in it. not adding more.")
conn.close()

In [22]:
def get_public_permissions(table_name):
    try:
        conn = psycopg2.connect(dbname='crimes_db', user='dq')
        with conn.cursor() as cursor:
            # Query for permissions granted to public on the table
            cursor.execute("""SELECT privilege_type
                            FROM information_schema.table_privileges
                            WHERE table_name = %s AND grantee = 'public';""", [table_name])
        # Fetch all permission types granted to public
            permissions = [row[0] for row in cursor.fetchall()]
            return permissions

    except psycopg2.Error as error:
        raise Exception("Error checking public permissions: %s".format(error)) from error

def get_group_permissions(group_name):
    try:
        with psycopg2.connect(dbname='crimes_db', user='dq') as connection:
            with connection.cursor() as cursor:
                cursor.execute("""SELECT grantee, privilege_type
                                FROM information_schema.table_privileges
                                WHERE grantee = %s;""", [group_name])
                return cursor.fetchall()
    except psycopg2.ERROR as error:
        raise Exception("Error checking group permissions: %s".format(error))

In [28]:
revoke_all = "REVOKE ALL ON SCHEMA public FROM public;"
revoke_db = "REVOKE ALL ON DATABASE crimes_db FROM public;"
# conn = psycopg2.connect(dbname='crimes_db', user='dq')
# cur = conn.cursor()
# cur.execute(revoke_all)
# cur.execute(revoke_db)
# print('revoked previlages')
# conn.commit()
# conn.close()
get_public_permissions('crimes.boston_crimes')

[]

In [27]:
# Create new permissions group
new_readonly = "CREATE GROUP readonly NOLOGIN;"
new_readwrite = "CREATE GROUP readwrite NOLOGIN;"
connect_readonly = "GRANT CONNECT ON DATABASE crimes_db TO readonly;"
connect_readwrite = "GRANT CONNECT ON DATABASE crimes_db TO readwrite;"
schema_readonly = "GRANT USAGE ON SCHEMA crimes TO readonly;"
schema_readwrite = "GRANT USAGE ON SCHEMA crimes TO readwrite;"
permissions_readonly = "GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;"
permissions_readwrite = "GRANT INSERT, SELECT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;"
updates = [new_readonly, new_readwrite, connect_readonly, connect_readwrite, schema_readonly, schema_readwrite, permissions_readonly, permissions_readwrite]
with psycopg2.connect(dbname='crimes_db', user='dq') as connection:
    with connection.cursor() as cursor:
        for permission in updates:
            cursor.execute(permission)
    connection.commit()
    print('wrote new permissions')

In [ ]:
# Create new users to play with database
with psycopg2.connect(dbname='crimes_db', user='dq') as connection:
    with connection.cursor() as cursor:
        cursor.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
        cursor.execute("GRANT readonly TO data_analyst;")
        cursor.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
        cursor.execute("GRANT readwrite TO data_scientist;")
        print("New users created and assigned to groups!")